In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime

### Race Schedule
- Scatter plot or distribution of lat and long or Scatter plot of lat vs engine failure or long vs engine failure

In [13]:
def load_race_schedule_data():
    return pd.read_csv('race_schedule.csv')

def f1_circuit_world_map(year=2024):
    df_race_schedule = load_race_schedule_data()
    df_filtered_by_year = df_race_schedule[df_race_schedule['season'] == year]

    fig = px.scatter_geo(df_filtered_by_year,
                         lat='location-lat',
                         lon='location-long',
                         hover_name='circuitName',
                         hover_data={
                             'raceName': True,
                             'location-locality': True,
                             'location-country': True,
                             'season': True,
                             'round': True,
                             'location-lat': False,
                             'location-long': False
                         },
                         title=f'F1 Circuits World Map - {year}',
                         projection='equirectangular')

    fig.update_traces(marker=dict(color='blue', size=10, opacity=0.85, line=dict(width=1, color='black')))
    
    fig.update_layout(
        geo=dict(
            showland=True,
            landcolor='lightgray',
            showocean=True,
            oceancolor='lightblue',
            showcountries=True,
            countrycolor='white',
            showframe=False,
            coastlinecolor='black',
            projection_scale=1.2
        ),
        title={
            'text': f'F1 Circuits World Map - {year}',
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=24, color='darkblue'),
        margin=dict(l=0, r=0, t=50, b=0),
        
        hoverlabel=dict(
            bgcolor="lightyellow",
            font_size=14,
            font_family="Arial",
            font_color="black"
        )
    )
    
    return fig

def races_by_continent():
    """Creates a scatter_geo plot showing the number of F1 races per continent."""
    df_race_schedule = load_race_schedule_data()

    country_to_continent = {
        'Australia': 'Oceania',
        'China': 'Asia',
        'Bahrain': 'Asia',
        'Russia': 'Europe',
        'Spain': 'Europe',
        'Monaco': 'Europe',
        'Canada': 'North America',
        'Azerbaijan': 'Europe',
        'Austria': 'Europe',
        'UK': 'Europe',
        'Hungary': 'Europe',
        'Belgium': 'Europe',
        'Italy': 'Europe',
        'Singapore': 'Asia',
        'Malaysia': 'Asia',
        'Japan': 'Asia',
        'USA': 'North America',
        'Mexico': 'South America',
        'Brazil': 'South America', 
        'UAE': 'Asia',
        'France': 'Europe',
        'Germany': 'Europe',
        'Portugal': 'Europe',
        'Turkey': 'Europe',
        'Netherlands': 'Europe', 
        'Qatar': 'Asia',
        'Saudi Arabia': 'Asia',
        'United States': 'North America'
    }

    df_race_schedule['continent'] = df_race_schedule['location-country'].map(country_to_continent)

    continent_race_count = df_race_schedule.groupby('continent').size().reset_index(name='race_count')
    total_races = continent_race_count['race_count'].sum()
    continent_race_count['percentage'] = (continent_race_count['race_count'] / total_races) * 100

    continent_geo = {
        'Europe': {'lat': 54.5260, 'long': 15.2551},
        'Asia': {'lat': 34.0479, 'long': 100.6197},
        'North America': {'lat': 47.875, 'long': -105.2551},
        'South America': {'lat': -8.7832, 'long': -55.4915},
        'Oceania': {'lat': -22.7359, 'long': 140.0188},
    }
    
    continent_race_count['lat'] = continent_race_count['continent'].map(lambda x: continent_geo[x]['lat'])
    continent_race_count['long'] = continent_race_count['continent'].map(lambda x: continent_geo[x]['long'])

    fig = px.scatter_geo(continent_race_count,
                         lat='lat',
                         lon='long',
                         hover_name='continent',
                         hover_data={'lat': False, 'long': False, 'race_count': True, 'percentage': ':.2f'},
                         size='race_count',
                         size_max=20,
                         color='race_count',
                         title='Number and Percentage of F1 Races by Continent (2017-2024)',
                         projection='equirectangular',
                         color_continuous_scale=px.colors.sequential.Plasma)

    fig.update_layout(
        geo=dict(
            showland=True,
            landcolor='lightgray',
            showocean=True,
            oceancolor='lightblue',
            showcountries=True,
            countrycolor='white',
            showframe=False,
            coastlinecolor='black',
            projection_scale=1.2
        ),
        title={
        'text': 'F1 Races in Different Continents (2017-2024)',
        'x': 0.40,
        'xanchor': 'center',
        'yanchor': 'top'
    },
        title_font=dict(size=24, color='darkblue'),
        margin=dict(l=0, r=0, t=50, b=0),
        hoverlabel=dict(
            bgcolor="lightyellow",
            font_size=14,
            font_family="Arial",
            font_color="black"
        )
    )

    return fig

def races_by_circuit(start_year=2017, end_year=2024):
    """Creates a bar chart showing the number of races by circuit for a specified year range."""
    df_race_schedule = load_race_schedule_data()

    df_filtered = df_race_schedule[(df_race_schedule['season'] >= start_year) & (df_race_schedule['season'] <= end_year)]
 
    circuit_race_count = df_filtered.groupby('circuitName').size().reset_index(name='count')
    circuit_race_count = circuit_race_count.sort_values(by='count', ascending=False)

    fig = px.bar(
        circuit_race_count, 
        x='count',
        y='circuitName',
        title=f'Races by Circuit ({start_year}-{end_year})',
        hover_data={'circuitName': True, 'count': True},
        labels={'count': 'Number of Races', 'circuitName': 'Circuit'},
        color='count',
        color_continuous_scale=px.colors.sequential.Viridis,
        text='count'
    )
    
    fig.update_layout(
        title={'x': 0.5, 'xanchor': 'center', 'font': dict(size=24, color='darkblue')},
        xaxis=dict(title='Number of Races', showgrid=False, tickfont=dict(size=14, color='black'), titlefont=dict(size=16, color='black')),
        yaxis=dict(title='Circuit', showgrid=False, tickfont=dict(size=14, color='black'), titlefont=dict(size=16, color='black')),
        plot_bgcolor='white',
        margin=dict(l=200, r=50, t=80, b=50),
        hoverlabel=dict(bgcolor="lightyellow", font_size=14, font_family="Arial", font_color="black"),
        height=800,
        coloraxis_showscale=False
    )

    fig.update_traces(textposition='outside')

    return fig

def races_by_country(start_year=2017, end_year=2024):
    """Creates a bar chart showing the number of races by country for a specified year range."""
    df_race_schedule = load_race_schedule_data()

    df_race_schedule = df_race_schedule[(df_race_schedule['season'] >= start_year) & (df_race_schedule['season'] <= end_year)]
    
    df_race_schedule['location-country'] = df_race_schedule['location-country'].replace({'United States': 'USA'})
    
    race_count_by_country = df_race_schedule.groupby('location-country').size().reset_index(name='count')
    race_count_by_country = race_count_by_country.sort_values(by='count', ascending=False)

    fig = px.bar(race_count_by_country, 
                 x='count',
                 y='location-country',
                 title=f'Races by Country ({start_year}-{end_year})',
                 hover_data={'location-country': True, 'count': True},
                 labels={'count': 'Number of Races', 'location-country': 'Country'},
                 color='count',
                 color_continuous_scale=px.colors.sequential.Viridis,
                 text='count')

    fig.update_layout(
        title={'x': 0.5, 'xanchor': 'center', 'font': dict(size=24, color='darkblue')},
        xaxis=dict(title='Number of Races', showgrid=False, tickfont=dict(size=14, color='black'), titlefont=dict(size=16, color='black')),
        yaxis=dict(title='Country', showgrid=False, tickfont=dict(size=14, color='black'), titlefont=dict(size=16, color='black')),
        plot_bgcolor='white',
        margin=dict(l=200, r=50, t=80, b=50),
        hoverlabel=dict(bgcolor="lightyellow", font_size=14, font_family="Arial", font_color="black"),
        height=800,
        coloraxis_showscale=False
    )

    fig.update_traces(textposition='outside')

    return fig

f1_circuit_world_map().show()
races_by_continent().show()
races_by_circuit().show()
races_by_country().show()

In [33]:
def load_constructor_standings_data():
    df = pd.read_csv('constructor_standings.csv')

    df['constructorId'] = df['constructorId'].replace({'alphatauri': 'rb'})
    df['constructorName'] = df['constructorName'].replace({'AlphaTauri': 'RB F1 Team'})
    
    return df

def plot_constructor_ranking_vs_year(start_year=2017, end_year=2024):
    """Creates a line chart for Constructor Ranking vs Year, excluding discontinued teams, within a specified year range."""
    
    df_constructor_standings = load_constructor_standings_data()

    discontinued_teams = ['toro_rosso', 'force_india', 'racing_point']
    df_constructor_standings = df_constructor_standings[~df_constructor_standings['constructorId'].isin(discontinued_teams)]

    df_constructor_standings = df_constructor_standings[
        (df_constructor_standings['season'] >= start_year) &
        (df_constructor_standings['season'] <= end_year)
    ]

    df_constructor_standings['season'] = df_constructor_standings['season'].astype(int)
    df_constructor_standings['position'] = df_constructor_standings['position'].astype(int)

    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
    }

    fig = px.line(
        df_constructor_standings,
        x='season',
        y='position',
        color='constructorId',
        labels={
            'season': 'Year',
            'position': 'Ranking',
            'constructorId': 'Constructor'
        },
        markers=True,
        color_discrete_map=brand_colors
    )

    fig.update_yaxes(autorange="reversed", tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    fig.update_layout(
        xaxis=dict(title='Year', tickfont=dict(size=12), tickvals=list(range(start_year, end_year + 1))),
        yaxis=dict(title='Ranking', tickfont=dict(size=12)),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

def plot_constructor_wins_vs_year(start_year=2017, end_year=2024):
    """Creates a line chart for Number of Wins vs Year within a specified range."""
    
    df_constructor_standings = load_constructor_standings_data()

    discontinued_teams = ['toro_rosso', 'force_india', 'racing_point']
    df_constructor_standings = df_constructor_standings[~df_constructor_standings['constructorId'].isin(discontinued_teams)]

    df_constructor_standings['season'] = df_constructor_standings['season'].astype(int)
    df_constructor_standings['wins'] = df_constructor_standings['wins'].astype(int)
    
    df_filtered = df_constructor_standings[(df_constructor_standings['season'] >= start_year) & 
                                           (df_constructor_standings['season'] <= end_year)]

    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
    }

    fig = px.line(
        df_filtered,
        x='season',
        y='wins',
        color='constructorId',
        labels={
            'season': 'Year',
            'wins': 'Number of Wins',
            'constructorId': 'Constructor'
        },
        markers=True,
        color_discrete_map=brand_colors
    )

    fig.update_layout(
        xaxis=dict(title='Year', tickfont=dict(size=12), tickvals=list(range(start_year, end_year + 1))),
        yaxis=dict(title='Number of Wins', tickfont=dict(size=12)),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

def plot_constructor_points_vs_year(start_year=2017, end_year=2024):
    """Creates a line chart for Points vs Year for each constructor within a specified range."""
    
    df_constructor_standings = load_constructor_standings_data()

    discontinued_teams = ['toro_rosso', 'force_india', 'racing_point']
    df_constructor_standings = df_constructor_standings[~df_constructor_standings['constructorId'].isin(discontinued_teams)]

    df_constructor_standings['season'] = df_constructor_standings['season'].astype(int)
    df_constructor_standings['points'] = df_constructor_standings['points'].astype(float)
    
    # Filter data within the selected year range
    df_filtered = df_constructor_standings[(df_constructor_standings['season'] >= start_year) & 
                                           (df_constructor_standings['season'] <= end_year)]

    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
    }

    fig = px.line(
        df_filtered,
        x='season',
        y='points',
        color='constructorId',
        labels={
            'season': 'Year',
            'points': 'Points',
            'constructorId': 'Constructor'
        },
        markers=True,
        color_discrete_map=brand_colors
    )

    fig.update_layout(
        xaxis=dict(title='Year', tickfont=dict(size=12), tickvals=list(range(start_year, end_year + 1))),
        yaxis=dict(title='Points', tickfont=dict(size=12)),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

def plot_constructor_points_distribution_per_year():
    """Creates a box plot for the distribution of constructor points per year."""
    
    df_constructor_standings = load_constructor_standings_data()

    discontinued_teams = ['toro_rosso', 'force_india', 'racing_point']
    df_constructor_standings = df_constructor_standings[~df_constructor_standings['constructorId'].isin(discontinued_teams)]

    df_constructor_standings['season'] = df_constructor_standings['season'].astype(int)
    df_constructor_standings['points'] = df_constructor_standings['points'].astype(float)

    fig = px.box(
        df_constructor_standings,
        x='season',
        y='points',
        labels={
            'season': 'Year',
            'points': 'Points'
        }
    )

    fig.update_layout(
        xaxis=dict(title='Year', tickfont=dict(size=12)),
        yaxis=dict(title='Points', tickfont=dict(size=12)),
        plot_bgcolor='white',
        margin=dict(l=50, r=50, t=50, b=50),
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

def calculate_win_percentage(df):
    """Calculates overall and year-by-year win percentage for constructors and filters out zero-win and discontinued teams."""
    
    discontinued_teams = ['toro_rosso', 'force_india', 'racing_point']
    df = df[~df['constructorId'].isin(discontinued_teams)]

    overall_wins = df.groupby('constructorId')['wins'].sum().reset_index()
    overall_wins['win_percentage'] = (overall_wins['wins'] / overall_wins['wins'].sum()) * 100

    overall_wins = overall_wins[overall_wins['win_percentage'] > 0]
    
    overall_wins = overall_wins.sort_values(by='win_percentage', ascending=True)

    yearly_wins = df.groupby(['season', 'constructorId'])['wins'].sum().reset_index()
    yearly_races = df.groupby('season')['wins'].sum().reset_index().rename(columns={'wins': 'total_wins_per_year'})
    yearly_wins = yearly_wins.merge(yearly_races, on='season')
    yearly_wins['win_percentage'] = (yearly_wins['wins'] / yearly_wins['total_wins_per_year']) * 100

    yearly_wins = yearly_wins[yearly_wins['win_percentage'] > 0]

    return overall_wins, yearly_wins

def plot_overall_win_percentage():
    """Creates a bar chart for overall win percentage per constructor, excluding zero-win and discontinued teams."""
    df_constructor_standings = load_constructor_standings_data()
    overall_wins, _ = calculate_win_percentage(df_constructor_standings)
    
    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
    }

    fig = px.bar(
        overall_wins,
        x='constructorId',
        y='win_percentage',
        labels={'constructorId': 'Constructor', 'win_percentage': 'Win Percentage (%)'},
        text='win_percentage',
        color='constructorId',
        color_discrete_map=brand_colors  # Apply the brand colors here
    )

    fig.update_layout(
        xaxis=dict(title='Constructor', tickfont=dict(size=12)),
        yaxis=dict(title='Win Percentage (%)', tickfont=dict(size=12)),
        margin=dict(l=50, r=50, t=50, b=50),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )
    
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

    return fig

def plot_yearly_win_percentage():
    """Creates a bar chart for yearly win percentage per constructor, excluding zero-win and discontinued teams."""
    df_constructor_standings = load_constructor_standings_data()
    _, yearly_wins = calculate_win_percentage(df_constructor_standings)

    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
    }
    
    fig = px.bar(
        yearly_wins,
        x='season',
        y='win_percentage',
        color='constructorId',
        barmode='group',
        labels={'season': 'Year', 'win_percentage': 'Win Percentage (%)', 'constructorId': 'Constructor'},
        color_discrete_map=brand_colors,
    )
    
    fig.update_layout(
        xaxis=dict(title='Year', tickfont=dict(size=12)),
        yaxis=dict(title='Win Percentage (%)', tickfont=dict(size=12)),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )
    
    fig.update_traces(
        hovertemplate='<b>Year:</b> %{x}<br>' +
                      '<b>Win Percentage:</b> %{y:.2f}%<extra></extra>',
        customdata=yearly_wins[['constructorId']].values
    )
    
    return fig

plot_constructor_ranking_vs_year().show()
plot_constructor_wins_vs_year().show()
plot_constructor_points_vs_year().show()
plot_constructor_points_distribution_per_year().show()
plot_overall_win_percentage().show()
plot_yearly_win_percentage().show()

## Driver Standing

In [5]:
def load_driver_standings_data():
    """Loads and updates the driver standings data."""
    df = pd.read_csv('driver_standings.csv')

    df['constructorId'] = df['constructorId'].replace({'alphatauri': 'rb'})
    df['constructorName'] = df['constructorName'].replace({'AlphaTauri': 'RB F1 Team'})
    
    return df

def plot_top_drivers_by_points(top=10):
    """Creates a bar chart for the top 10 drivers with the highest total points (2017-2024) using Viridis color scale."""
    
    df_driver_standings = load_driver_standings_data()

    df_driver_standings = df_driver_standings[df_driver_standings['season'].between(2017, 2024)]

    total_points_per_driver = df_driver_standings.groupby('driverId')['points'].sum().reset_index()

    top_drivers = total_points_per_driver.sort_values(by='points', ascending=False).head(top)

    fig = px.bar(
        top_drivers,
        x='driverId',
        y='points',
        labels={'driverId': 'Driver', 'points': 'Total Points'},
        text='points',
        color='points',
        color_continuous_scale='Viridis'
    )

    fig.update_layout(
        xaxis=dict(title='Driver', tickfont=dict(size=12), autorange='reversed'),
        yaxis=dict(title='Total Points', tickfont=dict(size=12)),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

    return fig

def plot_top_drivers_by_wins(top=10):
    """Creates a bar chart for the top 10 drivers with the most wins (2017-2024) using Viridis color scale."""
    
    df_driver_standings = load_driver_standings_data()

    total_wins_per_driver = df_driver_standings.groupby('driverId')['wins'].sum().reset_index()

    top_drivers = total_wins_per_driver.sort_values(by='wins', ascending=False).head(top)

    fig = px.bar(
        top_drivers,
        x='driverId',
        y='wins',
        labels={'driverId': 'Driver', 'wins': 'Number of Wins'},
        text='wins',
        color='wins',
        color_continuous_scale='Viridis'
    )

    fig.update_layout(
        xaxis=dict(title='Driver', tickfont=dict(size=12), autorange='reversed'),
        yaxis=dict(title='Number of Wins', tickfont=dict(size=12)),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

    return fig

def plot_driver_progression(driver_id):
    """Creates three visualizations for a specific driver: Points vs Year, Wins vs Year, Standing vs Year (reversed)."""
    
    df_driver_standings = load_driver_standings_data()

    df_driver = df_driver_standings[df_driver_standings['driverId'] == driver_id]

    fig_points = px.line(
        df_driver,
        x='season',
        y='points',
        labels={'season': 'Year', 'points': 'Points'},
        markers=True
    )
    fig_points.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        margin=dict(l=50, r=50, t=50, b=50),
    )

    fig_wins = px.line(
        df_driver,
        x='season',
        y='wins',
        labels={'season': 'Year', 'wins': 'Number of Wins'},
        markers=True
    )
    fig_wins.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        margin=dict(l=50, r=50, t=50, b=50),
    )

    fig_standing = px.line(
        df_driver,
        x='season',
        y='position',
        labels={'season': 'Year', 'position': 'Position'},
        markers=True
    )
    fig_standing.update_layout(
        yaxis=dict(autorange='reversed'),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        margin=dict(l=50, r=50, t=50, b=50),
    )

    return fig_points, fig_wins, fig_standing

plot_top_drivers_by_points().show()
plot_top_drivers_by_wins().show()
fig_points, fig_wins, fig_standing = plot_driver_progression('hamilton')
fig_points.show()
fig_wins.show()
fig_standing.show()

### Race Results


In [43]:
def load_race_results_data():
    """Loads the race results data from CSV."""
    df = pd.read_csv('race_results.csv')
    df['constructorId'] = df['constructorId'].replace({'alphatauri': 'rb'})
    discontinued_teams = ['toro_rosso', 'force_india', 'racing_point']
    df = df[~df['constructorId'].isin(discontinued_teams)]
    return df

def load_driver_info():
    """Loads the driver information from CSV."""
    df_driver_info = pd.read_csv('driver_info.csv')
    df_driver_info['dateOfBirth'] = pd.to_datetime(df_driver_info['dateOfBirth'])
    return df_driver_info

def get_merged_race_driver_data():
    """Merges race results with driver info and calculates driver age at each race."""
    df_race_results = load_race_results_data()
    df_driver_info = load_driver_info()
    
    df_merged = pd.merge(df_race_results, df_driver_info[['driverId', 'dateOfBirth']], on='driverId')
    
    df_merged['date'] = pd.to_datetime(df_merged['date'])
    
    df_merged['age_at_race'] = (df_merged['date'] - df_merged['dateOfBirth']).dt.days / 365
    
    return df_merged

def plot_grid_vs_position(season_range):
    """Creates a scatter plot showing the relationship between grid start and final position for a range of seasons."""
    
    df_race_results = load_race_results_data()

    df_filtered = df_race_results[(df_race_results['season'] >= season_range[0]) & (df_race_results['season'] <= season_range[1])]

    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
    }

    fig = px.scatter(
        df_filtered,
        x='grid',
        y='position',
        color='constructorId',
        hover_data=['driverId', 'raceName', 'points'],
        labels={'grid': 'Grid Start', 'position': 'Final Position'},
        color_discrete_map=brand_colors
    )

    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        xaxis=dict(title='Grid Start', range=[1, 20]),
        yaxis=dict(title='Final Position', autorange='reversed', range=[1, 20]),
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

def plot_mechanical_issues(selected_season_range):
    df_mechanical_issues = pd.read_csv('finishing_status_mechanica_issues_2017_2024.csv')
    total_occurrences = df_mechanical_issues['count'].sum()
    df_mechanical_issues['percentage'] = (df_mechanical_issues['count'] / total_occurrences) * 100
    threshold = 2.0
    df_mechanical_issues['status_grouped'] = df_mechanical_issues.apply(
        lambda row: row['status'] if row['percentage'] >= threshold else 'Other', axis=1
    )
    df_mechanical_issues_grouped = df_mechanical_issues.groupby('status_grouped').agg(
        {'count': 'sum', 'percentage': 'sum'}
        ).reset_index()
    df_top_mechanical_issues = df_mechanical_issues_grouped[(df_mechanical_issues_grouped['percentage'] >= 5.0) & 
                                                        (df_mechanical_issues_grouped['status_grouped'] != 'Other')]
    
    fig_pie = px.pie(df_mechanical_issues_grouped, 
             values='percentage', 
             names='status_grouped', 
             hole=0.5,
             hover_data=['count'],
             labels={'percentage':'Percentage'}
            )

    fig_pie.update_traces(textinfo='percent+label', textfont_size=12,
                      hovertemplate='<b>%{label}</b><br>Occurrences: %{customdata[0]}<br>Percentage: %{value:.2f}%')
    fig_pie.update_layout(showlegend=True, paper_bgcolor='lightgray')
    
    df_race_results = load_race_results_data()
    df_filtered = df_race_results[(df_race_results['season'] >= selected_season_range[0]) & 
                              (df_race_results['season'] <= selected_season_range[1])]
    top_issues = df_top_mechanical_issues['status_grouped'].tolist()
    df_filtered = df_filtered[df_filtered['status'].isin(top_issues)]
    df_grouped = df_filtered.groupby(['season', 'constructorId', 'status']).size().reset_index(name='count')
    df_pivot = df_grouped.pivot_table(index=['constructorId', 'season'], columns='status', values='count', fill_value=0).reset_index()
    df = pd.melt(df_pivot, id_vars=['constructorId', 'season'], var_name='issue', value_name='count')
    
    fig_stacked_bar = px.bar(df, 
                 x='constructorId', 
                 y='count', 
                 color='issue',
                 labels={'constructorId': 'Constructor', 'count': 'Issue Count'},
                 barmode='stack',
                 hover_data=['season'])
    
    fig_stacked_bar.update_layout(xaxis={'categoryorder': 'total descending'}, 
                      plot_bgcolor='white',
                      paper_bgcolor='lightgray')
    
    return fig_pie, fig_stacked_bar

def plot_avg_points_vs_age():
    """Plot average points per season against age."""
    df_driver_info = load_driver_info()
    df_merged = get_merged_race_driver_data()
    df_season_avg_points = df_merged.groupby(['driverId', 'season']).agg(
        avg_points=('points', 'mean'),
        first_race_date=('date', 'min'),
    ).reset_index()

    df_season_avg_points = pd.merge(df_season_avg_points, df_driver_info[['driverId', 'dateOfBirth']], on='driverId')
    df_season_avg_points['age_at_start_of_season'] = (pd.to_datetime(df_season_avg_points['first_race_date']) - 
                                                      pd.to_datetime(df_season_avg_points['dateOfBirth'])).dt.days / 365

    fig = px.scatter(df_season_avg_points, 
                     x='age_at_start_of_season', 
                     y='avg_points', 
                     labels={'age_at_start_of_season': 'Age at Start of Season', 'avg_points': 'Average Points'},
                     hover_data=['driverId', 'season'])
    fig.update_layout(margin=dict(l=50, r=50, t=80, b=50), showlegend=False, paper_bgcolor='lightgray')
    fig.update_traces(hovertemplate=(
        '<b>Driver ID:</b> %{customdata[0]}<br>' +
        '<b>Season:</b> %{customdata[1]}<br>' +
        '<b>Age at Start:</b> %{x:.1f}<br>' +
        '<b>Avg Point:</b> %{y:.1f}<br>'
    ))
    return fig

def plot_avg_position_vs_age():
    """Plot average position per season against age."""
    df_driver_info = load_driver_info()
    df_merged = get_merged_race_driver_data()
    df_season_avg_position = df_merged.groupby(['driverId', 'season']).agg(
        avg_position=('position', 'mean'),
        first_race_date=('date', 'min'),
    ).reset_index()

    df_season_avg_position = pd.merge(df_season_avg_position, df_driver_info[['driverId', 'dateOfBirth']], on='driverId')
    df_season_avg_position['age_at_start_of_season'] = (pd.to_datetime(df_season_avg_position['first_race_date']) - 
                                                        pd.to_datetime(df_season_avg_position['dateOfBirth'])).dt.days / 365

    fig = px.scatter(df_season_avg_position, 
                     x='age_at_start_of_season', 
                     y='avg_position', 
                     labels={'age_at_start_of_season': 'Age at Start of Season', 'avg_position': 'Average Position'},
                     hover_data=['driverId', 'season'])
    fig.update_yaxes(autorange='reversed', tickvals=[1, 5, 10, 15, 20])
    fig.update_layout(margin=dict(l=50, r=50, t=80, b=50), showlegend=False, paper_bgcolor='lightgray')
    fig.update_traces(hovertemplate=(
        '<b>Driver ID:</b> %{customdata[0]}<br>' +
        '<b>Season:</b> %{customdata[1]}<br>' +
        '<b>Age at Start:</b> %{x:.1f}<br>' +
        '<b>Avg Position:</b> %{y:.1f}<br>'
    ))
    return fig

def plot_avg_max_speed_vs_age():
    """Plot average max speed per season against age."""
    df_driver_info = load_driver_info()
    df_merged = get_merged_race_driver_data()
    df_season_avg_speed = df_merged.groupby(['driverId', 'season']).agg(
        avg_max_speed=('Max Avg Speed', 'mean'),
        first_race_date=('date', 'min'),
    ).reset_index()

    df_season_avg_speed = pd.merge(df_season_avg_speed, df_driver_info[['driverId', 'dateOfBirth']], on='driverId')
    df_season_avg_speed['age_at_start_of_season'] = (pd.to_datetime(df_season_avg_speed['first_race_date']) - 
                                                     pd.to_datetime(df_season_avg_speed['dateOfBirth'])).dt.days / 365

    fig = px.scatter(df_season_avg_speed, 
                     x='age_at_start_of_season',
                     y='avg_max_speed',
                     labels={'age_at_start_of_season': 'Age at Start of Season', 'avg_max_speed': 'Average Max Speed (km/h)'},
                     hover_data=['driverId', 'season'])
    
    fig.update_layout(margin=dict(l=50, r=50, t=80, b=50), showlegend=False, paper_bgcolor='lightgray')
    fig.update_traces(hovertemplate=(
        '<b>Driver ID:</b> %{customdata[0]}<br>' +
        '<b>Season:</b> %{customdata[1]}<br>' +
        '<b>Age at Start:</b> %{x:.1f}<br>' +
        '<b>Avg Max Speed:</b> %{y:.1f}<br>'
    ))
    return fig

def plot_top_drivers_podiums(top_n=6, season_range=(2017, 2024)):
    """Plots a stacked bar chart showing podium counts for the top drivers over a specified season range."""
    df_race_results = load_race_results_data()
    
    df_filtered = df_race_results[(df_race_results['season'] >= season_range[0]) & 
                                  (df_race_results['season'] <= season_range[1])]
    
    df_podiums = df_filtered[df_filtered['position'].isin([1, 2, 3])]

    df_podium_counts = df_podiums.groupby(['driverId', 'position']).size().reset_index(name='count')

    df_total_podiums = df_podium_counts.groupby('driverId')['count'].sum().reset_index(name='total_podiums')
    top_drivers = df_total_podiums.nlargest(top_n, 'total_podiums')['driverId'].tolist()

    df_top_podiums = df_podium_counts[df_podium_counts['driverId'].isin(top_drivers)]
    
    df_top_podiums['position'] = df_top_podiums['position'].map({1: '1st', 2: '2nd', 3: '3rd'})
    
    fig = px.bar(df_top_podiums, 
                 x='driverId', 
                 y='count', 
                 color='position',
                 title=f'Top {top_n} Drivers Podium Count ({season_range[0]} - {season_range[1]})',
                 labels={'driverId': 'Driver', 'count': 'Podium Count'},
                 category_orders={'position': ['1st', '2nd', '3rd']},
                 color_discrete_map={'1st': '#FFD700', '2nd': '#C0C0C0', '3rd': '#CD7F32'},
                 barmode='stack')

    fig.update_xaxes(categoryorder='total ascending')

    fig.update_layout(xaxis_title='Driver', yaxis_title='Podium Count', 
                      plot_bgcolor='white', paper_bgcolor='lightgray')

    return fig

def plot_top_constructors_podiums(top_n=6, season_range=(2017, 2024)):
    """Plots a stacked bar chart showing podium counts for the top constructors over a specified season range."""
    df_race_results = load_race_results_data()
    
    df_filtered = df_race_results[(df_race_results['season'] >= season_range[0]) & 
                                  (df_race_results['season'] <= season_range[1])]
    
    df_podiums = df_filtered[df_filtered['position'].isin([1, 2, 3])]

    df_podium_counts = df_podiums.groupby(['constructorId', 'position']).size().reset_index(name='count')

    df_total_podiums = df_podium_counts.groupby('constructorId')['count'].sum().reset_index(name='total_podiums')
    top_constructors = df_total_podiums.nlargest(top_n, 'total_podiums')['constructorId'].tolist()

    df_top_podiums = df_podium_counts[df_podium_counts['constructorId'].isin(top_constructors)]
    
    df_top_podiums['position'] = df_top_podiums['position'].map({1: '1st', 2: '2nd', 3: '3rd'})
    
    fig = px.bar(df_top_podiums, 
                 x='constructorId', 
                 y='count', 
                 color='position',
                 title=f'Top {top_n} Constructors Podium Count ({season_range[0]} - {season_range[1]})',
                 labels={'constructorId': 'Constructor', 'count': 'Podium Count'},
                 category_orders={'position': ['1st', '2nd', '3rd']},
                 color_discrete_map={'1st': '#FFD700', '2nd': '#C0C0C0', '3rd': '#CD7F32'},
                 barmode='stack')

    fig.update_xaxes(categoryorder='total ascending')

    fig.update_layout(xaxis_title='Constructor', yaxis_title='Podium Count', 
                      plot_bgcolor='white', paper_bgcolor='lightgray')

    return fig

plot_grid_vs_position((2023, 2023)).show()
fig1, fig2 = plot_mechanical_issues([2020, 2024])
fig1.show()
fig2.show()
plot_avg_points_vs_age().show()
plot_avg_position_vs_age().show()
plot_avg_max_speed_vs_age().show()
plot_top_drivers_podiums().show()
plot_top_constructors_podiums().show()

/var/folders/w0/zjn_yl050qn4kdvrggtg30sh0000gn/T/ipykernel_30221/2980562515.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



/var/folders/w0/zjn_yl050qn4kdvrggtg30sh0000gn/T/ipykernel_30221/2980562515.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



/var/folders/w0/zjn_yl050qn4kdvrggtg30sh0000gn/T/ipykernel_30221/2980562515.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



/var/folders/w0/zjn_yl050qn4kdvrggtg30sh0000gn/T/ipykernel_30221/2980562515.py:219: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/w0/zjn_yl050qn4kdvrggtg30sh0000gn/T/ipykernel_30221/2980562515.py:254: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
def plot_head_to_head_performance(df_race_results, year, constructor_id):
    """Plots head-to-head comparison of two drivers from the same constructor for a given year."""
    
    # Step 1: Filter data by year and constructor
    df_filtered = df_race_results[(df_race_results['season'] == year) & 
                                  (df_race_results['constructorId'] == constructor_id)]
    
    # Step 2: Identify the two drivers
    unique_drivers = df_filtered['driverId'].unique()
    if len(unique_drivers) < 2:
        return None, f"Less than 2 drivers found for {constructor_id} in {year}"
    
    driver1, driver2 = unique_drivers[:2]  # Assuming only two drivers
    
    # Step 3: Initialize counts for race and grid comparisons
    driver1_race_wins = driver2_race_wins = 0
    driver1_grid_wins = driver2_grid_wins = 0
    
    # Step 4: Loop through each race to compare positions
    for _, race in df_filtered.groupby('round'):
        race_data = race[['driverId', 'position', 'grid']]
        race_driver1 = race_data[race_data['driverId'] == driver1]
        race_driver2 = race_data[race_data['driverId'] == driver2]
        
        if not race_driver1.empty and not race_driver2.empty:
            # Compare race positions
            if race_driver1['position'].values[0] < race_driver2['position'].values[0]:
                driver1_race_wins += 1
            else:
                driver2_race_wins += 1
            
            # Compare grid positions
            if race_driver1['grid'].values[0] < race_driver2['grid'].values[0]:
                driver1_grid_wins += 1
            else:
                driver2_grid_wins += 1
    
    # Step 5: Prepare data for bar chart
    comparison_data = {
        'Metric': ['Race Position Wins', 'Race Position Wins', 'Grid Position Wins', 'Grid Position Wins'],
        'Driver': [driver1, driver2, driver1, driver2],
        'Count': [driver1_race_wins, driver2_race_wins, driver1_grid_wins, driver2_grid_wins]
    }
    df_comparison = pd.DataFrame(comparison_data)
    
    # Step 6: Plot the bar chart
    fig = px.bar(df_comparison, x='Metric', y='Count', color='Driver',
                 title=f'Head-to-Head Comparison: {driver1} vs {driver2} ({year})',
                 labels={'Count': 'Times Finished Ahead', 'Metric': 'Comparison'},
                 barmode='group')
    
    return fig, None

# Usage example
df_race_results = load_race_results_data()
fig, error = plot_head_to_head_performance(df_race_results, year=2024, constructor_id='williams')

if fig:
    fig.show()
else:
    print(error)

### Lap times

In [207]:
def load_lap_times_data():
    """Loads the lap times data from concatenated CSV files."""
    file_paths = [f'lap_times_{year}.csv' for year in range(2017, 2025)]
    dataframes = [pd.read_csv(file) for file in file_paths]
    lap_times_df = pd.concat(dataframes, ignore_index=True)
    return lap_times_df

def convert_time_to_seconds(time_str):
    """Converts lap time from string format 'M:SS.mmm' to seconds."""
    minutes, seconds = time_str.split(':')
    total_seconds = int(minutes) * 60 + float(seconds)
    return total_seconds

In [401]:
def plot_driver_lap_times(year, round_num, lower_percentile=5, upper_percentile=95):
    """Plots a box plot of lap times for all drivers in a given race, filtering out extreme outliers."""
    
    # Load the lap times and race schedule data
    lap_times_df = load_lap_times_data()
    race_schedule_df = pd.read_csv('race_schedule.csv')  # Assuming race_schedule.csv is available
    
    # Filter the race schedule to get the race name
    race_name = race_schedule_df[(race_schedule_df['season'] == year) & 
                                 (race_schedule_df['round'] == round_num)]['raceName'].values[0]
    
    # Filter lap times for the specific year and round
    lap_times_df = lap_times_df[(lap_times_df['season'] == year) & (lap_times_df['round'] == round_num)]
    
    # Convert the time column to seconds
    lap_times_df['lap_time_seconds'] = lap_times_df['time'].apply(convert_time_to_seconds)
    
    # Calculate lap time limits based on percentiles
    lower_limit = lap_times_df['lap_time_seconds'].quantile(lower_percentile / 100)
    upper_limit = lap_times_df['lap_time_seconds'].quantile(upper_percentile / 100)
    
    # Filter lap times to exclude outliers
    lap_times_filtered = lap_times_df[
        (lap_times_df['lap_time_seconds'] >= lower_limit) &
        (lap_times_df['lap_time_seconds'] <= upper_limit)
    ]
    
    # Create the box plot
    fig = px.box(
        lap_times_filtered,
        x='driverId',
        y='lap_time_seconds',
        title=f'Filtered Lap Times per Driver - {race_name}, Season {year}, Round {round_num}',
        labels={'lap_time_seconds': 'Lap Time (seconds)', 'driverId': 'Driver'}
    )
    
    # Update layout for better readability
    fig.update_layout(
        xaxis_title='Driver',
        yaxis_title='Lap Time (seconds)',
        title_x=0.5,
        title_font=dict(size=20),
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )
    
    return fig

plot_driver_lap_times(2024, 3, 10, 90)

In [161]:
def impute_lap_times_lap_count_based(driver_id, season, round_num, df_lap_times, df_race_results, race_schedule):
    """Imputes missing lap times based on lap count, ensuring the same Grand Prix is used for previous seasons."""
    
    # Load driver lap times for the given race
    driver_lap_times = df_lap_times[(df_lap_times['driverId'] == driver_id) &
                                    (df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]
    
    # Get the maximum number of laps in this race round
    max_laps_in_race = df_lap_times[(df_lap_times['season'] == season) & 
                                    (df_lap_times['round'] == round_num)]['lap'].max()
    
    # Get the maximum number of laps for this driver
    max_laps_for_driver = driver_lap_times['lap'].max()
    
    # Filter the race schedule by season, round, and raceName
    race_schedule = race_schedule[['season', 'round', 'raceName']]
    
    # Load the race name for the current season and round
    current_race_name = race_schedule[(race_schedule['season'] == season) &
                                      (race_schedule['round'] == round_num)]['raceName'].values[0]
    
    # Proceed with imputation if the driver did not complete all laps
    if max_laps_for_driver < max_laps_in_race:
        # Get lap times from previous seasons for this same Grand Prix
        previous_lap_times = df_lap_times.merge(race_schedule, on=['season', 'round'])
        previous_lap_times = previous_lap_times[(previous_lap_times['driverId'] == driver_id) &
                                                (previous_lap_times['raceName'] == current_race_name) &
                                                (previous_lap_times['season'] < season)]
        
        # Convert the time column to seconds
        previous_lap_times['lap_time_seconds'] = previous_lap_times['time'].apply(convert_time_to_seconds)
        
        if not previous_lap_times.empty:
            median_lap_time = previous_lap_times['lap_time_seconds'].median()
            
            # Identify missing laps in the current race
            current_laps = driver_lap_times['lap'].unique()
            missing_laps = set(range(1, max_laps_in_race + 1)) - set(current_laps)
            
            # Impute missing laps with the median lap time
            for lap in missing_laps:
                new_lap = {
                    'season': season,
                    'round': round_num,
                    'lap': lap,
                    'position': None,
                    'driverId': driver_id,
                    'time': median_lap_time
                }
                df_lap_times = df_lap_times.append(new_lap, ignore_index=True)
    
    return df_lap_times

In [239]:
from statsmodels.tsa.arima.model import ARIMA

def impute_lap_times_arima(driver_id, season, round_num, df_lap_times, df_race_results, race_schedule):
    """Imputes missing lap times based on a time series ARIMA model, with outlier filtering."""
    
    # Get available lap times for the given race and driver
    driver_lap_times = df_lap_times[(df_lap_times['driverId'] == driver_id) &
                                    (df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]
    
    max_laps_in_race = df_lap_times[(df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]['lap'].max()
    max_laps_for_driver = driver_lap_times['lap'].max()
    
    # Ensure the same Grand Prix in previous seasons
    current_race_name = race_schedule[(race_schedule['season'] == season) &
                                      (race_schedule['round'] == round_num)]['raceName'].values[0]
    previous_lap_times = df_lap_times.merge(race_schedule[['season', 'round', 'raceName']], on=['season', 'round'])
    previous_lap_times = previous_lap_times[(previous_lap_times['driverId'] == driver_id) &
                                            (previous_lap_times['raceName'] == current_race_name) &
                                            (previous_lap_times['season'] <= season)]
    
    # Convert time column to seconds
    previous_lap_times['lap_time_seconds'] = previous_lap_times['time'].apply(convert_time_to_seconds)
    
    # Filter out extreme lap times based on lower and upper percentiles
    lower_percentile = 1
    upper_percentile = 99
    lower_limit = previous_lap_times['lap_time_seconds'].quantile(lower_percentile / 100)
    upper_limit = previous_lap_times['lap_time_seconds'].quantile(upper_percentile / 100)
    
    previous_lap_times_filtered = previous_lap_times[
        (previous_lap_times['lap_time_seconds'] >= lower_limit) &
        (previous_lap_times['lap_time_seconds'] <= upper_limit)
    ]
    
    if max_laps_for_driver < max_laps_in_race and not previous_lap_times_filtered.empty:
        available_lap_times = previous_lap_times['lap_time_seconds'].reset_index(drop=True)

        # Train the ARIMA model (p, d, q) need tuning based on data
        model = ARIMA(available_lap_times, order=(10, 1, 0))
        model_fit = model.fit()

        # Forecast the number of laps we are missing
        n_forecasts = max_laps_in_race - max_laps_for_driver
        forecasted_laps = model_fit.forecast(steps=n_forecasts)

        # Impute the missing lap times
        missing_laps = sorted(set(range(1, max_laps_in_race + 1)) - set(driver_lap_times['lap'].unique()))
        imputed_laps = pd.DataFrame([{
                'season': season,
                'round': round_num,
                'lap': lap,
                'position': None,
                'driverId': driver_id,
                'lap_time_seconds': forecast
            } for lap, forecast in zip(missing_laps, forecasted_laps)])

        # Concatenate imputed laps with driver lap times
        combined_driver_lap_times = pd.concat([driver_lap_times, imputed_laps], ignore_index=True)
        # Concatenate the historical lap times with current and imputed lap times
        final_combined_lap_times = pd.concat([previous_lap_times, combined_driver_lap_times], ignore_index=True)

        return final_combined_lap_times

    # If no imputation is needed or no historical data is available, return driver_lap_times and previous_lap_times combined
    return pd.concat([previous_lap_times, driver_lap_times], ignore_index=True)

driver_id = 'stroll'
season = 2024
round_num = 2
df_lap_times = load_lap_times_data()
df_race_results = load_race_results_data()
race_schedule = load_race_schedule_data()
df = impute_lap_times_arima(driver_id, season, round_num, df_lap_times, df_race_results, race_schedule)

lower_percentile=1
upper_percentile=99

lower_limit = df['lap_time_seconds'].quantile(lower_percentile / 100)
upper_limit = df['lap_time_seconds'].quantile(upper_percentile / 100)

df = df[
        (df['lap_time_seconds'] >= lower_limit) &
        (df['lap_time_seconds'] <= upper_limit)
    ]

x_values = pd.Series(range(len(df)))

fig = px.scatter(df, x=x_values, y='lap_time_seconds',
                 title='Lap Times Scatter Plot', labels={'x': 'Lap Number', 'lap_time_seconds': 'Lap Time (seconds)'})
fig.show()

In [231]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def impute_lap_times_exponential_smoothing(driver_id, season, round_num, df_lap_times, race_schedule):
    """Imputes missing lap times based on an Exponential Smoothing model, with outlier filtering."""
    
    # Get available lap times for the given race and driver
    driver_lap_times = df_lap_times[(df_lap_times['driverId'] == driver_id) &
                                    (df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]
    
    max_laps_in_race = df_lap_times[(df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]['lap'].max()
    max_laps_for_driver = driver_lap_times['lap'].max()
    
    # Ensure the same Grand Prix in previous seasons
    current_race_name = race_schedule[(race_schedule['season'] == season) &
                                      (race_schedule['round'] == round_num)]['raceName'].values[0]
    previous_lap_times = df_lap_times.merge(race_schedule[['season', 'round', 'raceName']], on=['season', 'round'])
    previous_lap_times = previous_lap_times[(previous_lap_times['driverId'] == driver_id) &
                                            (previous_lap_times['raceName'] == current_race_name) &
                                            (previous_lap_times['season'] <= season)]
    
    # Convert time column to seconds
    previous_lap_times['lap_time_seconds'] = previous_lap_times['time'].apply(convert_time_to_seconds)
    
    # Filter out extreme lap times based on lower and upper percentiles
    lower_percentile = 1
    upper_percentile = 99
    lower_limit = previous_lap_times['lap_time_seconds'].quantile(lower_percentile / 100)
    upper_limit = previous_lap_times['lap_time_seconds'].quantile(upper_percentile / 100)
    
    previous_lap_times_filtered = previous_lap_times[
        (previous_lap_times['lap_time_seconds'] >= lower_limit) &
        (previous_lap_times['lap_time_seconds'] <= upper_limit)
    ]
    
    if max_laps_for_driver < max_laps_in_race and not previous_lap_times_filtered.empty:
        available_lap_times = previous_lap_times_filtered['lap_time_seconds'].reset_index(drop=True)

        # Train the Exponential Smoothing model (additive trend for more stable lap times)
        model = ExponentialSmoothing(available_lap_times, trend='add', seasonal=None)
        model_fit = model.fit()

        # Forecast the number of laps we are missing
        n_forecasts = max_laps_in_race - max_laps_for_driver
        forecasted_laps = model_fit.forecast(steps=n_forecasts)

        # Impute the missing lap times
        missing_laps = sorted(set(range(1, max_laps_in_race + 1)) - set(driver_lap_times['lap'].unique()))
        imputed_laps = pd.DataFrame([{
                'season': season,
                'round': round_num,
                'lap': lap,
                'position': None,
                'driverId': driver_id,
                'lap_time_seconds': forecast
            } for lap, forecast in zip(missing_laps, forecasted_laps)])

        # Concatenate imputed laps with driver lap times
        combined_driver_lap_times = pd.concat([driver_lap_times, imputed_laps], ignore_index=True)
        # Concatenate the historical lap times with current and imputed lap times
        final_combined_lap_times = pd.concat([previous_lap_times, combined_driver_lap_times], ignore_index=True)

        return final_combined_lap_times

    # If no imputation is needed or no historical data is available, return driver_lap_times and previous_lap_times combined
    return pd.concat([previous_lap_times, driver_lap_times], ignore_index=True)


In [391]:
def impute_lap_times_mean_median(driver_id, season, round_num, df_lap_times, race_schedule, method='mean'):
    """Imputes missing lap times using mean or median imputation for a specific driver, season, and round."""
    
    # Ensure the round number corresponds to the correct race name
    race_name = race_schedule[(race_schedule['season'] == season) &
                              (race_schedule['round'] == round_num)]['raceName'].values[0]
    
    # Collect historical lap times for the driver in the same race across all seasons
    historical_lap_times = df_lap_times.merge(race_schedule[['season', 'round', 'raceName']], on=['season', 'round'])
    historical_data = historical_lap_times[(historical_lap_times['driverId'] == driver_id) &
                                           (historical_lap_times['raceName'] == race_name) &
                                           (historical_lap_times['season'] < season)].copy()  # Make a copy here
    
    # Convert time to seconds
    historical_data.loc[:, 'lap_time_seconds'] = historical_data['time'].apply(convert_time_to_seconds)
    
    # Filter out extreme lap times based on lower and upper percentiles
    lower_limit = historical_data['lap_time_seconds'].quantile(0.01)
    upper_limit = historical_data['lap_time_seconds'].quantile(0.99)
    
    filtered_data = historical_data[(historical_data['lap_time_seconds'] >= lower_limit) &
                                    (historical_data['lap_time_seconds'] <= upper_limit)]
    
    # Determine the imputation value based on the selected method
    if method == 'mean':
        impute_value = filtered_data['lap_time_seconds'].mean()
    elif method == 'median':
        impute_value = filtered_data['lap_time_seconds'].median()
    else:
        raise ValueError("Invalid method specified. Use 'mean' or 'median'.")
    
    # Get available lap times for the given race and driver
    driver_lap_times = df_lap_times[(df_lap_times['driverId'] == driver_id) &
                                    (df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]
    driver_lap_times = driver_lap_times.copy()
    driver_lap_times.loc[:, 'lap_time_seconds'] = driver_lap_times['time'].apply(convert_time_to_seconds)
    
    max_laps_in_race = df_lap_times[(df_lap_times['season'] == season) &
                                    (df_lap_times['round'] == round_num)]['lap'].max()
    current_laps = driver_lap_times['lap'].unique()
    missing_laps = sorted(set(range(1, max_laps_in_race + 1)) - set(current_laps))
    
    # Impute the missing laps with the chosen imputation value
    imputed_laps = pd.DataFrame([{
        'season': season,
        'round': round_num,
        'lap': lap,
        'position': None,  # Position unknown for imputed laps
        'driverId': driver_id,
        'lap_time_seconds': impute_value
    } for lap in missing_laps])
    
    # Combine the imputed laps with the existing lap times
    combined_driver_lap_times = pd.concat([driver_lap_times, imputed_laps], ignore_index=True)
    
    return combined_driver_lap_times


In [400]:
driver_id = 'sainz'
season = 2024
round_num = 9
df = impute_lap_times_mean_median(driver_id, season, round_num, df_lap_times, race_schedule, method='median')


### Pit stop

In [415]:
import pandas as pd
import plotly.express as px

def plot_avg_pit_stop_duration_by_circuit(year):
    """
    Plots the average pit stop duration by circuit for a given year.
    
    Parameters:
        year (int): The year for which to plot average pit stop durations.
    """
    df_pit_results = pd.read_csv('pit_results.csv')
    df_race_results = load_race_results_data()
    race_schedule = pd.read_csv('race_schedule.csv')
    
    df_race_results_reduced = df_race_results[['season', 'round', 'driverId', 'constructorId']]
    df_pit_results = df_pit_results.merge(df_race_results_reduced, on=['season', 'round', 'driverId'], how='left')
    
    df_pit_results_with_circuits = df_pit_results.merge(
        race_schedule[['season', 'round', 'circuitName']],
        on=['season', 'round'],
        how='left'
    )
    
    df_pit_results_year = df_pit_results_with_circuits[df_pit_results_with_circuits['season'] == year]
    
    avg_pit_stop_duration = df_pit_results_year.groupby('circuitName')['duration'].mean().reset_index()
    
    fig = px.bar(
        avg_pit_stop_duration,
        x='circuitName',
        y='duration',
        title=f'Average Pit Stop Duration by Circuit - {year}',
        labels={'circuitName': 'Circuit', 'duration': 'Avg. Duration (s)'},
        color='duration',
        color_continuous_scale=px.colors.sequential.Viridis,
        hover_data={'duration': ':.2f'}
    )
    
    fig.update_layout(
        xaxis={'categoryorder': 'total ascending'},
        title_x=0.5,
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

fig = plot_avg_pit_stop_duration_by_circuit(2023)
fig.show()


In [423]:
import pandas as pd
import plotly.express as px

def plot_pit_stop_duration_by_constructor(year):

    # Load data
    df_pit_results = pd.read_csv('pit_results.csv')
    df_race_results = load_race_results_data()

    # Merge df_pit_results with constructorId from race results
    df_race_results_reduced = df_race_results[['season', 'round', 'driverId', 'constructorId']]
    df_pit_results = df_pit_results.merge(df_race_results_reduced, on=['season', 'round', 'driverId'], how='left')

    # Filter for the given year
    df_pit_results_year = df_pit_results[df_pit_results['season'] == year]

    # Calculate mean pit stop duration for sorting
    constructor_mean_durations = df_pit_results_year.groupby('constructorId')['duration'].mean()
    sorted_constructors = constructor_mean_durations.sort_values(ascending=False).index.tolist()

    # Define the constructor brand colors
    brand_colors = {
        'mercedes': '#565F64',
        'ferrari': '#ff0000',
        'williams': '#00A0DE',
        'renault': '#FFFF00',
        'haas': '#E6002B',
        'mclaren': '#FF8000',
        'sauber': '#90EE90',
        'alfa': '#000000',
        'alphatauri': '#00008B',
        'alpine': '#C71585',
        'aston_martin': '#006400',
        'red_bull': '#FDD900',
        'rb': '#20394C'
        }
    
    # Plot the data with sorted constructors
    fig = px.box(
        df_pit_results_year,
        x='constructorId',
        y='duration',
        title=f'Pit Stop Duration by Constructor - {year}',
        labels={'constructorId': 'Constructor', 'duration': 'Pit Stop Duration (seconds)'},
        category_orders={'constructorId': sorted_constructors},  # Sort x-axis by mean duration
        color='constructorId',
        color_discrete_map=brand_colors,
        hover_data={'duration': ':.2f'}
    )
    
    # Customize layout
    fig.update_layout(
        xaxis_title='Constructor',
        yaxis_title='Pit Stop Duration (seconds)',
        title_x=0.5,
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig

fig = plot_pit_stop_duration_by_constructor(2023)
fig.show()


In [430]:
import pandas as pd
import plotly.express as px

def plot_pit_stop_count_by_lap(year=None, round_num=None):
    """Plots a histogram showing the count of pit stops by lap for a given year and round."""
    # Load data
    df_pit_results = pd.read_csv('pit_results.csv')

    # Filter by year and round if specified
    if year:
        df_pit_results = df_pit_results[df_pit_results['season'] == year]
    if round_num:
        df_pit_results = df_pit_results[df_pit_results['round'] == round_num]

    # Plot the histogram
    fig = px.histogram(
        df_pit_results,
        x='lap',
        title=f'Pit Stop Distribution by Lap - {year if year else "All Years"} {f"Round {round_num}" if round_num else ""}',
        labels={'lap': 'Lap Number', 'count': 'Pit Stop Count'},
        nbins=25  # Adjust bin size for readability
    )

    # Customize layout
    fig.update_layout(
        xaxis_title='Lap Number',
        yaxis_title='Pit Stop Count',
        title_x=0.5,
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )

    return fig
fig = plot_pit_stop_count_by_lap(2024, 5)
fig.show()